In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 3.58G/3.58G [00:27<00:00, 139MB/s]


In [ ]:
train ='/content/fruits-fresh-and-rotten-for-classification/dataset/test'
validation_dir = '/content/fruits-fresh-and-rotten-for-classification/dataset/train'

In [ ]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [ ]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

6 Classes
2698 Train images
10901 Test images


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =224,224
input_shape=(img_width,img_height,3)
batch_size =32
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 2698 images belonging to 6 classes.
Found 10901 images belonging to 6 classes.


In [ ]:
train_datagen


In [ ]:
od.download('https://www.kaggle.com/datasets/keras/inceptionresnetv2')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 392M/392M [00:07<00:00, 56.5MB/s]


In [ ]:
from keras import Model
from keras import optimizers
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dense

model_dir = "../content/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
model = Sequential()
model.add( tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights= model_dir, input_shape=(224, 244, 3)))

model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 6, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 6)                 9222      
                                                                 
Total params: 54,345,958
Trainable params: 54,285,414
Non-trainable params: 60,544
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=2,shuffle=True,callbacks=early_stopping_callback)

Epoch 1/2
85/85 [==============================] - 117s 1s/step - loss: 0.0620 - accuracy: 0.9785 - val_loss: 0.3946 - val_accuracy: 0.8933
Epoch 2/2
85/85 [==============================] - 117s 1s/step - loss: 0.0828 - accuracy: 0.9733 - val_loss: 2.3252 - val_accuracy: 0.7353


In [ ]:
model.save('inceptioResnetv2_model.h5')

In [ ]:
model.evaluate(test_generator, verbose=1)

341/341 [==============================] - 65s 190ms/step - loss: 2.3252 - accuracy: 0.7353


[2.3252334594726562, 0.735253632068634]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = load_model('/content/drive/MyDrive/Fresh rotten/CNN_model.h5')

In [ ]:
model.evaluate(test_generator, verbose=1)

341/341 [==============================] - 51s 148ms/step - loss: 0.3341 - accuracy: 0.8789


[0.3340620696544647, 0.8789101839065552]